In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import vectorbt as vbt
import talib



In [2]:
end_time = datetime.now()
start_time = end_time - timedelta(days = 59)

data = yf.download("^NSEI",start = start_time,end = end_time,interval="5m")




[*********************100%***********************]  1 of 1 completed


In [6]:
hammer = talib.CDLHAMMER(data.Open, data.High, data.Low, data.Close)
print(hammer[hammer == 100])

Datetime
2023-05-31 13:50:00+05:30    100
2023-05-31 14:55:00+05:30    100
2023-06-01 10:00:00+05:30    100
2023-06-01 12:10:00+05:30    100
2023-06-01 12:30:00+05:30    100
                            ... 
2023-07-26 13:40:00+05:30    100
2023-07-26 14:40:00+05:30    100
2023-07-28 09:20:00+05:30    100
2023-07-28 12:05:00+05:30    100
2023-07-28 12:15:00+05:30    100
Length: 96, dtype: int32


In [7]:
help(talib.CDLHAMMER)

Help on function CDLHAMMER in module talib._ta_lib:

CDLHAMMER(...)
    CDLHAMMER(open, high, low, close)
    
    Hammer (Pattern Recognition)
    
    Inputs:
        prices: ['open', 'high', 'low', 'close']
    Outputs:
        integer (values are -100, 0 or 100)



In [8]:
hanging_man = talib.CDLHANGINGMAN(data.Open, data.High, data.Low, data.Close)
print(hanging_man[hanging_man == -100])

Datetime
2023-05-31 13:30:00+05:30   -100
2023-06-01 09:45:00+05:30   -100
2023-06-01 09:50:00+05:30   -100
2023-06-01 11:35:00+05:30   -100
2023-06-02 15:05:00+05:30   -100
2023-06-05 15:25:00+05:30   -100
2023-06-06 10:55:00+05:30   -100
2023-06-06 13:25:00+05:30   -100
2023-06-07 09:45:00+05:30   -100
2023-06-07 13:55:00+05:30   -100
2023-06-08 10:40:00+05:30   -100
2023-06-08 13:00:00+05:30   -100
2023-06-12 10:20:00+05:30   -100
2023-06-12 11:10:00+05:30   -100
2023-06-13 09:40:00+05:30   -100
2023-06-15 09:50:00+05:30   -100
2023-06-15 15:05:00+05:30   -100
2023-06-21 09:55:00+05:30   -100
2023-06-21 13:05:00+05:30   -100
2023-06-22 12:30:00+05:30   -100
2023-06-23 11:15:00+05:30   -100
2023-06-27 14:35:00+05:30   -100
2023-06-27 15:05:00+05:30   -100
2023-06-28 09:30:00+05:30   -100
2023-06-30 09:35:00+05:30   -100
2023-06-30 14:45:00+05:30   -100
2023-07-05 11:55:00+05:30   -100
2023-07-05 13:55:00+05:30   -100
2023-07-06 13:00:00+05:30   -100
2023-07-07 09:25:00+05:30   -100
2

In [9]:
buys = (hammer == 100)
sells = (hanging_man == -100)

In [10]:
buys

Datetime
2023-05-31 09:15:00+05:30    False
2023-05-31 09:20:00+05:30    False
2023-05-31 09:25:00+05:30    False
2023-05-31 09:30:00+05:30    False
2023-05-31 09:35:00+05:30    False
                             ...  
2023-07-28 15:05:00+05:30    False
2023-07-28 15:10:00+05:30    False
2023-07-28 15:15:00+05:30    False
2023-07-28 15:20:00+05:30    False
2023-07-28 15:25:00+05:30    False
Length: 3150, dtype: bool

In [32]:
pf = vbt.Portfolio.from_signals(data.Close,buys,sells, fees = 0.005)

In [33]:
print(pf.stats())

C:\Users\Pranav\AppData\Roaming\Python\Python39\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

C:\Users\Pranav\AppData\Roaming\Python\Python39\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

C:\Users\Pranav\AppData\Roaming\Python\Python39\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

C:\Users\Pranav\AppData\Roaming\Python\Python39\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2023-05-30 09:15:00+05:30
End                           2023-07-27 15:25:00+05:30
Period                                             3150
Start Value                                       100.0
End Value                                     76.069902
Total Return [%]                             -23.930098
Benchmark Return [%]                           5.821176
Max Gross Exposure [%]                            100.0
Total Fees Paid                               27.531423
Max Drawdown [%]                              24.292675
Max Drawdown Duration                            3135.0
Total Trades                                         32
Total Closed Trades                                  31
Total Open Trades                                     1
Open Trade PnL                                -0.723601
Win Rate [%]                                        0.0
Best Trade [%]                                -0.144164
Worst Trade [%]                               -1

In [34]:
pf.plot().show()

In [36]:
window_length = 240
returns = []

for x in range(0,len(data) - window_length, 24):
    window_data = data.iloc[x:x+window_length]
    hammer = talib.CDLHAMMER(window_data.Open, window_data.High, 
                             window_data.Low, window_data.Close)
    hanging_man = talib.CDLHANGINGMAN(window_data.Open, window_data.High, 
                                      window_data.Low, window_data.Close)
    buys = (hammer == 100)
    sells = (hanging_man == -100)
    pf = vbt.Portfolio.from_signals(window_data.Close,buys,sells, fees = 0.005)
    returns.append(pf.total_return())
    
print(sum(returns)/len(returns))


-0.020785050117632594
